In [4]:
# competition_name = "iamsouravbanerjee/animal-image-dataset-90-different-animals"

# kaggle_creds_path = "./kaggle.json"

# ! pip install kaggle --quiet


# ! mkdir ~/.kaggle
# ! cp /content/kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kagg|le.json

# ! kaggle datasets download -d {competition_name}

# ! mkdir kaggle_data

mkdir: cannot create directory ‘/root/.kaggle’: File exists

/bin/bash: line 1: le.json: command not found

chmod: cannot access '/root/.kaggle/kagg': No such file or directory



 99% 650M/656M [00:04<00:00, 222MB/s]

100% 656M/656M [00:04<00:00, 142MB/s]

mkdir: cannot create directory ‘kaggle_data’: File exists


In [ ]:
# ! unzip animal-image-dataset-90-different-animals.zip -d kaggle_data

In [3]:
import os
import shutil

original_parent_dir = "/kaggle/input/animal-image-dataset-90-different-animals/animals/animals"
new_parent_dir = "./onevrest"
target_folder = "f2"
special_folder = "badger"

os.makedirs(new_parent_dir, exist_ok=True)

# Copy the special folder as-is
shutil.copytree(os.path.join(original_parent_dir, special_folder),
                os.path.join(new_parent_dir, special_folder))

# Process the other folders efficiently
target_dir = os.path.join(new_parent_dir, target_folder)
os.makedirs(target_dir, exist_ok=True)

for dir in os.listdir(original_parent_dir):
    dir_path = os.path.join(original_parent_dir, dir)
    if os.path.isdir(dir_path) and dir != special_folder:
        for file in os.listdir(dir_path):
            file_src = os.path.join(dir_path, file)
            file_dest = os.path.join(target_dir, file)
            shutil.copy2(file_src, file_dest)

        # Do NOT remove the source directories


In [3]:
import os
import shutil

# Original parent directory containing the 90 folders
original_parent_dir = "/kaggle/input/animal-image-dataset-90-different-animals/animals/animals"

# New parent directory where the 5 folders (d1 to d5) will be created
new_parent_dir = "./fiveclass"

# Make sure the new parent directory exists
os.makedirs(new_parent_dir, exist_ok=True)

# Get a sorted list of all folders in the original parent directory
# Assuming folder names allow them to be correctly sorted to reflect the desired order
folders = sorted([f for f in os.listdir(original_parent_dir) if os.path.isdir(os.path.join(original_parent_dir, f))])

# Calculate the number of folders to distribute into each of the 5 new folders
folders_per_group = len(folders) // 5

# Function to copy files from source to destination
def copy_files(src_dir, dest_dir):
    os.makedirs(dest_dir, exist_ok=True)
    for file in os.listdir(src_dir):
        shutil.copy2(os.path.join(src_dir, file), os.path.join(dest_dir, file))

# Iterate through each of the original folders and copy its contents to the correct new folder
for i, folder in enumerate(folders):
    # Determine the index for the new folder (d1 to d5)
    new_folder_index = i // folders_per_group + 1
    if new_folder_index > 5:  # Ensure we don't go beyond d5
        new_folder_index = 5

    # Construct the new folder name and path
    new_folder_name = f"d{new_folder_index}"
    new_folder_path = os.path.join(new_parent_dir, new_folder_name)

    # Copy the contents of the current folder to the new folder
    current_folder_path = os.path.join(original_parent_dir, folder)
    copy_files(current_folder_path, new_folder_path)

print("All folders have been processed and images distributed into d1 to d5.")


All folders have been processed and images distributed into d1 to d5.


In [7]:
import os
import torch
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim

In [6]:
# Transformations
transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load dataset
data_dir = 'onevrest'
data_dir2='fiveclass'


In [6]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=3, shuffle=True, random_state=42)

In [9]:
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, Subset
from torchvision import models
from torch import nn, optim
import torch
from sklearn.metrics import confusion_matrix
import numpy as np

num_classes = 2  # Adjust this based on your dataset
num_epochs = 2
batch_size = 32

n_splits = 3  # Number of folds

full_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

all_acc = []
all_conf_matrices = []

for fold, (train_idx, test_idx) in enumerate(kfold.split(full_dataset)):
    # Splitting datasets per fold
    train_subset = Subset(full_dataset, train_idx)
    test_subset = Subset(full_dataset, test_idx)

    # Data loaders for the current fold
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_subset, batch_size=batch_size, shuffle=False)

    # Reinitialize model and optimizer
    model = models.convnext_tiny(weights=None)
    num_ftrs = model.classifier[2].in_features
    model.classifier[2] = nn.Linear(num_ftrs, num_classes)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    # Training loop for current fold
    for epoch in range(num_epochs):
        model.train()
        for i, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

    # Evaluation for current fold
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    acc = (np.array(y_true) == np.array(y_pred)).mean()
    all_acc.append(acc)
    conf_matrix = confusion_matrix(y_true, y_pred)
    all_conf_matrices.append(conf_matrix)
    print(f'Fold {fold+1}, Test Accuracy: {acc:.4f}')

# Average accuracy and confusion matrix
average_acc = np.mean(all_acc)
average_conf_matrix = np.mean(all_conf_matrices, axis=0)
print(f'Average Test Accuracy: {average_acc:.4f}')
print('Average Confusion Matrix:')
print(average_conf_matrix)


Fold 1, Test Accuracy: 0.9900
Fold 2, Test Accuracy: 0.9883
Fold 3, Test Accuracy: 0.9883
Average Test Accuracy: 0.9889
Average Confusion Matrix:
[[   0.           20.        ]
 [   0.         1779.66666667]]


In [ ]:
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, Subset
from torchvision import models
from torch import nn, optim
import torch
from sklearn.metrics import confusion_matrix
import numpy as np

num_classes = 2  # Adjust this based on your dataset
num_epochs = 2
batch_size = 32

n_splits = 3  # Number of folds

full_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

all_acc = []
all_conf_matrices = []

for fold, (train_idx, test_idx) in enumerate(kfold.split(full_dataset)):
    # Splitting datasets per fold
    train_subset = Subset(full_dataset, train_idx)
    test_subset = Subset(full_dataset, test_idx)

    # Data loaders for the current fold
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_subset, batch_size=batch_size, shuffle=False)

    # Reinitialize model and optimizer
    model = models.resnet18(pretrained=False)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    # Training loop for current fold
    for epoch in range(num_epochs):
        model.train()
        for i, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

    # Evaluation for current fold
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    acc = (np.array(y_true) == np.array(y_pred)).mean()
    all_acc.append(acc)
    conf_matrix = confusion_matrix(y_true, y_pred)
    all_conf_matrices.append(conf_matrix)
    print(f'Fold {fold+1}, Test Accuracy: {acc:.4f}')

# Average accuracy and confusion matrix
average_acc = np.mean(all_acc)
average_conf_matrix = np.mean(all_conf_matrices, axis=0)
print(f'Average Test Accuracy: {average_acc:.4f}')
print('Average Confusion Matrix:')
print(average_conf_matrix)


In [ ]:
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, Subset
from torchvision import models
from torch import nn, optim
import torch
from sklearn.metrics import confusion_matrix
import numpy as np

num_classes = 5  # Adjust this based on your dataset
num_epochs = 3
batch_size = 32

n_splits = 3  # Number of folds

full_dataset = datasets.ImageFolder(root=data_dir2, transform=transform)
print(full_dataset)

kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

all_acc = []
all_conf_matrices = []

for fold, (train_idx, test_idx) in enumerate(kfold.split(full_dataset)):
    # Splitting datasets per fold
    train_subset = Subset(full_dataset, train_idx)
    test_subset = Subset(full_dataset, test_idx)

    # Data loaders for the current fold
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_subset, batch_size=batch_size, shuffle=False)

    # Reinitialize model and optimizer
    model = models.convnext_tiny(weights='IMAGENET1K_V1')
    num_ftrs = model.classifier[2].in_features
    model.classifier[2] = nn.Linear(num_ftrs, num_classes)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    # Training loop for current fold
    for epoch in range(num_epochs):
        model.train()
        for i, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

    # Evaluation for current fold
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    acc = (np.array(y_true) == np.array(y_pred)).mean()
    all_acc.append(acc)
    conf_matrix = confusion_matrix(y_true, y_pred)
    all_conf_matrices.append(conf_matrix)
    print(f'Fold {fold+1}, Test Accuracy: {acc:.4f}')

# Average accuracy and confusion matrix
average_acc = np.mean(all_acc)
average_conf_matrix = np.mean(all_conf_matrices, axis=0)
print(f'Average Test Accuracy: {average_acc:.4f}')
print('Average Confusion Matrix:')
print(average_conf_matrix)


Dataset ImageFolder
    Number of datapoints: 5400
    Root location: fiveclass
    StandardTransform
Transform: Compose(
               RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear, antialias=warn)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )


Downloading: "https://download.pytorch.org/models/convnext_tiny-983f1562.pth" to /root/.cache/torch/hub/checkpoints/convnext_tiny-983f1562.pth
100%|██████████| 109M/109M [00:07<00:00, 16.1MB/s] 


Fold 1, Test Accuracy: 0.1950


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class ComplexCustomCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(ComplexCustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.conv4 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(512 * 28 * 28, 1024)  # Adjusted for 224x224 input images
        self.bn_fc1 = nn.BatchNorm1d(1024)
        self.fc2 = nn.Linear(1024, 512)
        self.bn_fc2 = nn.BatchNorm1d(512)
        self.fc3 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = x.view(-1, 512 * 28 * 28)  # Adjusted for 224x224 input images
        x = self.dropout(F.relu(self.bn_fc1(self.fc1(x))))
        x = self.dropout(F.relu(self.bn_fc2(self.fc2(x))))
        x = self.fc3(x)
        return x

# Example usage
num_classes = 10  # Adjust based on your dataset
model = ComplexCustomCNN(num_classes=num_classes)
print(model)


ComplexCustomCNN(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=401408, out_features=1024, bias=True)
  (bn_fc1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (

In [9]:
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from torch import nn, optim
import torch
from sklearn.metrics import confusion_matrix
import numpy as np



# Define the transformation for the dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

num_epochs = 10
batch_size = 32
n_splits = 3
num_classes = 5

full_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

all_acc = []
all_conf_matrices = []

for fold, (train_idx, test_idx) in enumerate(kfold.split(full_dataset)):
    train_subset = Subset(full_dataset, train_idx)
    test_subset = Subset(full_dataset, test_idx)

    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_subset, batch_size=batch_size, shuffle=False)

    model = ComplexCustomCNN(num_classes=num_classes)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        model.train()
        for i, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    acc = (np.array(y_true) == np.array(y_pred)).mean()
    all_acc.append(acc)
    conf_matrix = confusion_matrix(y_true, y_pred)
    all_conf_matrices.append(conf_matrix)
    print(f'Fold {fold+1}, Test Accuracy: {acc:.4f}')

average_acc = np.mean(all_acc)
average_conf_matrix = np.mean(all_conf_matrices, axis=0)
print(f'Average Test Accuracy: {average_acc:.4f}')
print('Average Confusion Matrix:')
print(average_conf_matrix)


ValueError: Expected input batch_size (8) to match target batch_size (32).